# Выбор локации для скважины

По пробам нефти в трёх регионах (в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов) необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль, то есть, где бурить новую скважину. Анализ возможной прибыли и риски выполним техникой *Bootstrap.*



## Загрузка и подготовка данных

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Данные геологоразведки трёх регионов находятся в файлах:

In [ ]:
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')

In [ ]:
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')

In [ ]:
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

**Описание данных**

* id — уникальный идентификатор скважины;
* f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
* product — объём запасов в скважине (тыс. баррелей).

In [ ]:
geo_data_0

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [ ]:
geo_data_1

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


In [ ]:
geo_data_2

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


In [ ]:
geo_data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
geo_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
geo_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Пропусков нет. 

In [ ]:
geo_data_0.duplicated().sum()

0

In [ ]:
geo_data_1.duplicated().sum()

0

In [ ]:
geo_data_2.duplicated().sum()

0

Явных дубликатов нет.

## Обучение и проверка модели для каждого региона

In [ ]:
def geo_data_model(geo_data):
    geo_data = geo_data.drop(['id'], axis=1) # для дальнейшего исследования и обучения модели удалим столбец 'id'
    
    features = geo_data.drop(['product'], axis=1) # извлечем признаки
    target = geo_data['product'] # извлечем целевой признак
    
    # отделим 25% данных для валидационной выборки
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345) 
    # features_train - признаки, тренировочная выборка
    # target_train - целевой признак, тренировочная выборка
    # features_valid - признаки, валидационная выборка
    # target_valid - целевой признак, валидационная выборка
    
    model = LinearRegression() # инициализируем модель LinearRegression
    model.fit(features_train, target_train) # обучим модель на тренировочной выборке
    predictions_valid = model.predict(features_valid) # получим предсказания модели на валидационной выборке
    
    result = mean_squared_error(target_valid, predictions_valid) ** 0.5 # значение метрики RMSE на валидационной выборке

    predicted_valid = pd.Series(predictions_valid) # предсказания на валидационной выборке
    target_valid = target_valid.reset_index(drop=True)  #правильные ответы на валидационной выборке
    predicted_valid_mean = predicted_valid.mean() # средний запас предсказанного сырья
    
    return predicted_valid, target_valid, predicted_valid_mean, result

In [ ]:
predicted_valid_0, target_valid_0, predicted_valid_mean_0, RMSE_0 = geo_data_model(geo_data_0) # 1 регион

In [ ]:
predicted_valid_1, target_valid_1, predicted_valid_mean_1, RMSE_1 = geo_data_model(geo_data_1) # 2 регион

In [ ]:
predicted_valid_2, target_valid_2, predicted_valid_mean_2, RMSE_2 = geo_data_model(geo_data_2) # 3 регион

Выведем средний запас предсказанного сырья и RMSE модели по регионам.

In [ ]:
result_geo_data = pd.DataFrame({
    'Регион':[1, 2, 3],
    'Средний запас предсказанного сырья':[predicted_valid_mean_0, predicted_valid_mean_1, predicted_valid_mean_2],
    'RMSE модели':[RMSE_0, RMSE_1, RMSE_2]
})

In [ ]:
result_geo_data

,Регион,Средний запас предсказанного сырья,RMSE модели
0,1,92.592568,37.579422
1,2,68.728547,0.893099
2,3,94.965046,40.029709


Величина RMSE (англ. root mean squared error, «корень из средней квадратичной ошибки»).

Лучшее (наименьшее) значение RMSE получено для 2-го региона, при этом средний запас предсказанного сырья самый меньший.

Однако для 1-го и 3-го регионов значения RMSE очень велики. Модели на данных 1-го и 3-го регионов предсказывают гораздо хуже.

## Подготовка к расчёту прибыли

3.1. Все ключевые значения для расчётов сохраним в отдельных переменных.

In [ ]:
BUDGET = 10000000000 # Бюджет на разработку скважин в регионе — 10 млрд рублей
COUNT_MAX = 500 # при разведке региона исследуют 500 точек
COUNT = 200 # с помощью машинного обучения выбирают 200 лучших для разработки
REVENUE_ONE_PRODUCT = 450000 # доход с каждой единицы продукта 450 тыс. рублей
B = 1000 # количество выборок для техники Bootstrap

3.2. Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. 

In [ ]:
revenue_one_well = BUDGET / COUNT / REVENUE_ONE_PRODUCT

In [ ]:
revenue_one_well # минимальное среднее количество продукта в месторождениях региона, достаточное для разработки

111.11111111111111

Полученный средний объём сырья, достаточный для безубыточной разработки, превышает средние запасы в каждом из регионов. По средним запасам предсказанного сырья нет возможности выделить лучший регион для возможной разработки новых скважин.

## Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели

* Выбираем скважины с максимальными значениями предсказаний.
* Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
* Рассчитаем прибыль для полученного объёма сырья.

In [ ]:
# функция для подсчета прибыли
def revenue(target, predicted, count, revenue_one_product):
    predicted_sorted = predicted.sort_values(ascending=False)
    selected = target[predicted_sorted.index][:count]
    return revenue_one_product * selected.sum() - BUDGET

## Расчёт прибыли и рисков 

Посчитаем риски и прибыль для каждого региона:
* Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
* Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.

In [ ]:
def revenue_Bootstrap(target, predicted, count, revenue_one_product, BUDGET):
    state = np.random.RandomState(12345)
    values = []

    for _ in range(B):
        target_subsample = target.sample(n=COUNT_MAX, replace=True, random_state=state) 
        predicted_subsample = predicted[target_subsample.index] 
        val = revenue(target_subsample, predicted_subsample, count, revenue_one_product)
        values.append(val) 
   
    values = pd.Series(values)

#    values.quantile(0.025) - 2.5%-квантиль 
#    values.quantile(0.975) - 97.5%-квантиль  
#    values.mean() - средняя прибыль 
#    (values < 0).mean() - риск

    return values.mean(), values.quantile(0.025), values.quantile(0.975), (values < 0).mean()

In [ ]:
mean_0, lower_0, upper_0, risk_0 = revenue_Bootstrap(
    target_valid_0, predicted_valid_0, COUNT, REVENUE_ONE_PRODUCT, BUDGET)

In [ ]:
mean_1, lower_1, upper_1, risk_1 = revenue_Bootstrap(
    target_valid_1, predicted_valid_1, COUNT, REVENUE_ONE_PRODUCT, BUDGET)

In [ ]:
mean_2, lower_2, upper_2, risk_2 = revenue_Bootstrap(
    target_valid_2, predicted_valid_2, COUNT, REVENUE_ONE_PRODUCT, BUDGET)

In [ ]:
result_geo_data['Средняя прибыль'] = [mean_0, mean_1, mean_2]
result_geo_data['95%-й доверительный интервал'] = [[lower_0, upper_0], [lower_1, upper_1], [lower_2, upper_2]]
result_geo_data['Риск убытков, %'] = [risk_0 * 100, risk_1 * 100, risk_2 * 100]

In [ ]:
result_geo_data

,Регион,Средний запас предсказанного сырья,RMSE модели,Средняя прибыль,95%-й доверительный интервал,"Риск убытков, %"
0,1,92.592568,37.579422,4.259385e+08,"[-102090094.83793654, 947976353.358369]",6.0
1,2,68.728547,0.893099,5.152228e+08,"[68873225.37050177, 931547591.2570496]",1.0
2,3,94.965046,40.029709,4.350084e+08,"[-128880547.32978901, 969706954.1802679]",6.4


In [ ]:
mean_1

515222773.44328994

**Общий вывод** 

В качестве региона для разработки скважин считаю возможным предложить 2-ой регион, в связи с тем, что по результатам исследования у этого региона:
* наименьшая вероятность убытков: 1% (меньше 2.5%)
* наибольшая средняя прибыль: 515 222 773.44 
    